In [1]:
%load_ext autoreload
# %reload_ext autoreload

# Reload all modules imported with %aimport every time before executing the Python code typed
%autoreload 1

%aimport context_nn 
%aimport phrase_feeder 
%aimport notes
%aimport watch_point
%aimport cluster
import numpy as np
from bitarray import bitarray
from context_nn import ContextNN
from watch_point import WatchPoint
from cluster import Cluster
from phrase_feeder import PhraseFeeder
from pprint import pprint
import math
import constants as const

In [5]:
abc_notes = notes.Notes(note_count=26, 
                        notation_count=10, 
                        active_bits=8, 
                        bit_count=255)

In [2]:
import pickle

def load_phrase_base(file_name: str) -> (dict, list):
    with open(file_name, 'rb') as f:
        data = pickle.load(f)
        phrase_base = data.get('phrase_base', {})
        marks = data.get('marks', {})
        return phrase_base, marks

In [3]:
phrase_base, marks = load_phrase_base('./data/texts/phrase_base.pickle')

In [4]:
from ipythonblocks import BlockGrid

def draw_notation(notation: np.array):
    bit_grid = BlockGrid(len(notation), 1, fill=(17, 41, 129))
    for block in range(bit_grid.width):
        color = bit_grid[0, block]
        if notation[block]:
            bit_grid[0, block] = (244, 195, 173)
    bit_grid.lines_on = False
    bit_grid.show()

def draw_note_notations(notes, note_idx: int):
    for i in range(len(notes[note_idx])):
        draw_notation(notes.note_notation_as_bits(note_idx, i))

In [6]:
key = list(phrase_base.keys())[0]
phrase = phrase_base[key][0]
bit_chord = abc_notes.phrase_chord(phrase)
draw_notation(abc_notes.notation_as_bits(bit_chord))
print(phrase)
for note_idx, notation_idx in phrase:
    notation = abc_notes.note_notation_as_bits(note_idx, notation_idx)
    draw_notation(notation)

,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


[(8, 0), (0, 1), (13, 2), (24, 3), (18, 4)]


,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [96]:
feeder = PhraseFeeder(phrase_base, marks)    

In [ ]:
phrases, output_bits = feeder.take_phrase_bunch()
print(output_bits, marks[output_bits])
print(phrases)

In [74]:
abc_notes = notes.Notes(note_count=26, 
                        notation_count=10, 
                        active_bits=8, 
                        bit_count=255)

feeder = phrase_feeder.PhraseFeeder(phrase_base, marks)  

cxnn = context_nn.ContextNN(input_bit_count=255,
                            output_bit_count=8,
                            watch_point_count=100,
                            watch_bit_count=32,
                            cluster_make_threshold=6,
                            cluster_activate_threshold=4)

In [99]:
len(cxnn.watch_points.keys())

100

In [273]:
%%time
phrases, output_bits = feeder.take_phrase_batch(count=200)
output_bits = set(output_bits)
for phrase in phrases:
    bit_chord = abc_notes.phrase_chord(phrase)
    cxnn.receive_bits(input_bits=bit_chord, output_bits=output_bits)
    

Wall time: 1.63 s


In [40]:
def feed_phrase_batch(feeder, count=200):
    phrases, output_bits = feeder.take_phrase_batch(count=200)
    output_bits = set(output_bits)
    for phrase in phrases:
        bit_chord = abc_notes.phrase_chord(phrase)
        cxnn.receive_bits(input_bits=bit_chord, output_bits=output_bits)

def feed_n_batches(feeder, n=10, batch_size=200):
    for i in range(n):
        feed_phrase_batch(feeder, count=batch_size)
        print(f'batch {i+1}/{n}')
    print(f'clusters: {cxnn.cluster_count()}')
        
def show_point_stats(cxnn: ContextNN):
    cluster_counts = [wp.cluster_count() for wp in cxnn.watch_points.values()]
    output_bits = [(wp.output_bit, len(wp.clusters.values())) 
                   for wp in cxnn.watch_points.values()]
    print('Cluster count:', sum(cluster_counts))
    print(output_bits)
  

In [6]:
cxnn.state = const.STATE_LEARN

In [100]:
cxnn.state = const.STATE_CONSOLIDATE

In [75]:
%%time
feed_n_batches(feeder, n=10, batch_size=200)

batch 1/10
batch 2/10
batch 3/10
batch 4/10
batch 5/10
batch 6/10
batch 7/10
batch 8/10
batch 9/10
batch 10/10
clusters: 22321
Wall time: 14.1 s


In [12]:
show_point_stats(cxnn)
marks

Cluster count: 21784
[(5, 93), (4, 112), (4, 83), (4, 133), (5, 122), (3, 201), (4, 174), (1, 321), (0, 163), (0, 224), (5, 151), (7, 203), (6, 320), (0, 227), (5, 142), (7, 217), (4, 76), (0, 241), (4, 110), (7, 259), (7, 198), (3, 152), (4, 117), (1, 390), (0, 257), (1, 299), (7, 240), (7, 229), (7, 277), (0, 339), (1, 243), (0, 268), (6, 570), (1, 399), (7, 267), (2, 193), (7, 128), (2, 173), (6, 399), (6, 451), (4, 83), (3, 266), (6, 561), (1, 359), (2, 267), (2, 131), (0, 256), (6, 344), (4, 93), (4, 71), (0, 215), (7, 234), (4, 117), (4, 117), (1, 217), (4, 139), (6, 269), (4, 93), (4, 103), (4, 148), (4, 113), (2, 180), (0, 266), (5, 81), (2, 176), (6, 399), (2, 225), (2, 163), (5, 149), (5, 151), (4, 144), (7, 164), (4, 104), (4, 104), (1, 208), (3, 201), (0, 182), (0, 278), (2, 305), (5, 183), (5, 137), (4, 96), (1, 369), (0, 254), (6, 364), (2, 295), (5, 138), (2, 176), (5, 134), (0, 284), (4, 81), (3, 235), (1, 411), (2, 209), (3, 239), (6, 401), (1, 239), (6, 345), (7, 236)

{(0, 1, 3, 5): 'ukr',
 (0, 1, 6, 7): 'blg',
 (0, 3, 5, 6): 'epo',
 (0, 5, 6, 7): 'bel',
 (1, 2, 3, 6): 'pol',
 (1, 2, 4, 7): 'jbo',
 (1, 4, 6, 7): 'eng',
 (2, 3, 4, 5): 'rus'}

In [29]:
def dict_value(d: dict, n=0) -> any:
    i = 0
    diter = iter(d)
    while i <= n:
        key = next(diter)
        i += 1
    return d[key]

In [76]:
point = dict_value(cxnn.watch_points)
cluster = dict_value(point.clusters, 1)
pprint(cluster.stats)
# pprint(cluster.component_stats())
# pprint(cluster.bit_rate())

print(cluster.bit_mask)
sorted(cluster.component_stats().items(), key=lambda x: x[1], reverse=True)

# parts = cluster.component_stats()
# enthropy = 0.0
# for part in parts.values():
#     enthropy += -part * math.log2(part)
# print(enthropy)

{'00000000000000100000000000110010': 1,
 '00000000000011000000000000110010': 3,
 '00000000000011100000000000010000': 3,
 '00000000000101000000000000010010': 3,
 '00000000000101000000000000100010': 1,
 '00000000000101100000000000010000': 5,
 '00000000000111000000000000100000': 2,
 '00000000000111100000000000110010': 1}
bitarray('00000000000111100000000000110010')


[('00000000000101100000000000010000', 0.2631578947368421),
 ('00000000000101000000000000010010', 0.15789473684210525),
 ('00000000000011100000000000010000', 0.15789473684210525),
 ('00000000000011000000000000110010', 0.15789473684210525),
 ('00000000000111000000000000100000', 0.10526315789473684),
 ('00000000000111100000000000110010', 0.05263157894736842),
 ('00000000000000100000000000110010', 0.05263157894736842),
 ('00000000000101000000000000100010', 0.05263157894736842)]

In [32]:
noize_clusters = 0
total_clusters = 0
cluster_lens = {}
cluster_acts = {}
for wp in cxnn.watch_points.values():
    for cluster in wp.clusters.values():
        total_clusters += 1
        cluster_lens[len(cluster.bits)] = cluster_lens.get(len(cluster.bits), 0) + 1
        cluster_acts[len(cluster.stats)] = cluster_acts.get(len(cluster.stats), 0) + 1
        if not cluster.has_big_component(threshold=0.2, min_activations=10):
            noize_clusters += 1
print(total_clusters)
print(noize_clusters)
pprint(cluster_lens)
pprint(cluster_acts)


AttributeError: 'Cluster' object has no attribute 'bits'

In [26]:
my_cluster = Cluster((10, 20, 30), bitarray(10), 4)
print(type(my_cluster))
my_cluster.bits
my_cluster.stat_parts()

<class 'cluster.Cluster'>


{}